In [4]:
####*****************************************etape 1 Observation des resultats de SPERT et des vrais resultats par phrase
import json
import numpy as np
import pandas as pd
import glob
entities_types =["No","Loc","Org", "Peop","Other"]
relations_types =["Work_For", "Kill", "OrgBased_In", "Live_In", "Located_In"]



print(type(json))
with open("CONLL04\conll04_test.json") as file :
    data_reals = json.load(file)
with open("CONLL04/logs/spert_treshold_0.4/predictions_valid_epoch_16.json") as file :
    data_preds = json.load(file)
with open("CONLL04/logs/spert_treshold_0.4/neurASP-SUP_predictions_valid_epoch_16.json") as file :
    data_neur = json.load(file)  
    
def study (data_reals,data_preds):
    longn1 =0
    longn2 =0
    longn3 =0
    longr3 =0
    longr2 =0
    longr1 =0
    longnr3 =0
    longnr2 =0
    longnr1 =0
    longr =0
    longn=0
    longnr=0
    for doc_id , doc_real in enumerate(data_reals):
        #doc_id = 217
        entities_pred ={}
        entities_real ={}    
        for entity in doc_real["entities"]:
            entity_pos= (entity["start"],entity["end"])
            entity_words = (' '.join(doc_real["tokens"][entity["start"]:entity["end"]]))
            entities_real[str(entity_pos)] = (entity_words, entity["type"].lower())
        for entity in data_preds[doc_id]["entities"]:
            entity_pos= (entity["start"],entity["end"])
            entity_words = (' '.join(doc_real["tokens"][entity["start"]:entity["end"]]))
            entity_probs = entity["probs"].split("[")[1].split("]")[0]
            probs =np.fromstring(entity_probs, dtype=float, sep=' ')
            prob =probs[entities_types.index(entity['type'])]
            entities_pred[str(entity_pos)] =(prob , (entity_words, entity["type"].lower()) )

        for idx in entities_real:
            if idx not in entities_pred:
                entities_pred[idx]=(0,"0***")
        for idx in entities_pred:
            if idx not in entities_real:
                entities_real[idx]=(0,"0***")
        display_real=[]
        display_pred =[]
        positions=[]
        for idx in entities_real :
            display_real.append(entities_real[idx])
            display_pred.append(entities_pred[idx])
            positions.append(idx)
            longn+=1
            if entities_real[idx]!=entities_pred[idx][1] and entities_real[idx][1]!="0***"  and entities_pred[idx][1]!="0***" :
                longn3+=1
            if entities_real[idx]!=entities_pred[idx][1] and  entities_pred[idx][1]=="0***" :
                longn2+=1  # il n'identifie pas l'entité
            if entities_real[idx]!=entities_pred[idx][1] and  entities_real[idx][1]=="0***" :
                longn1+=1  # il identifie l'entité 
        df ={"position":positions , "real":display_real , "predictions":display_pred}
        df =pd.DataFrame(data=df)
        print("doc n°:",doc_id)
        print(df)

        
        relations_pred ={}
        relations_real ={}    
        for relation in doc_real["relations"]:
            relation_pos= (doc_real["entities"][relation["head"]]["start"], doc_real["entities"][relation["head"]]["end"]
                           ,doc_real["entities"][relation["tail"]]["start"],doc_real["entities"][relation["tail"]]["end"])
            relation_words = (' '.join(doc_real["tokens"][doc_real["entities"][relation["head"]]["start"]:doc_real["entities"][relation["head"]]["end"]]),
                           ' '.join(doc_real["tokens"][doc_real["entities"][relation["tail"]]["start"]:doc_real["entities"][relation["tail"]]["end"]]))
            relations_real[str(relation_pos)] = (relation_words, relation["type"].lower())
        for relation in data_preds[doc_id]["relations"]:

            relation_pos= (data_preds[doc_id]["entities"][relation["head"]]["start"], data_preds[doc_id]["entities"][relation["head"]]["end"]
                           ,data_preds[doc_id]["entities"][relation["tail"]]["start"],data_preds[doc_id]["entities"][relation["tail"]]["end"])
            relation_words = (' '.join(doc_real["tokens"][data_preds[doc_id]["entities"][relation["head"]]["start"]:data_preds[doc_id]["entities"][relation["head"]]["end"]]),
                   ' '.join(doc_real["tokens"][data_preds[doc_id]["entities"][relation["tail"]]["start"]:data_preds[doc_id]["entities"][relation["tail"]]["end"]]))
            relation_probs = relation["probs"].split("[")[1].split("]")[0]
            probs =np.fromstring(relation_probs, dtype=float, sep=' ')
            prob =probs[relations_types.index(relation['type'])]
            relations_pred[str(relation_pos)] = (prob, relation_words, relation["type"].lower())

        for idx in relations_real:
            if idx not in relations_pred:
                relations_pred[idx]=(0,"0***")
        for idx in relations_pred:
            if idx not in relations_real:
                relations_real[idx]=(0,"0***")
        display_real=[]
        display_pred =[]
        positions=[]
        for idx in relations_real :
            display_real.append(relations_real[idx])
            display_pred.append(relations_pred[idx])
            positions.append(idx)
            longr+=1
            if relations_real[idx]!=relations_pred[idx][1:] and relations_real[idx][1]!="0***"  and relations_pred[idx][1]!="0***" :
                longr3+=1
            if relations_real[idx]!=relations_pred[idx][1:] and  relations_pred[idx][1]=="0***" :
                longr2+=1  # il n'identifie pas l'entité
            if relations_real[idx]!=relations_pred[idx][1:] and  relations_real[idx][1]=="0***" :
                longr1+=1  # il identifie l'entité 
                
        df ={"position":positions , "real":display_real , "predictions":display_pred}
        df =pd.DataFrame(data=df)
        print("doc n°:",doc_id)
        print(df)
        
        
        
        relations_pred ={}
        relations_real ={}    
        for relation in doc_real["relations"]:
            relation_pos= (doc_real["entities"][relation["head"]]["start"], doc_real["entities"][relation["head"]]["end"]
                           ,doc_real["entities"][relation["tail"]]["start"],doc_real["entities"][relation["tail"]]["end"])
            relation_words = (' '.join(doc_real["tokens"][doc_real["entities"][relation["head"]]["start"]:doc_real["entities"][relation["head"]]["end"]]),
                           ' '.join(doc_real["tokens"][doc_real["entities"][relation["tail"]]["start"]:doc_real["entities"][relation["tail"]]["end"]]))
            type_entity1= doc_real["entities"][relation["head"]]["type"]
            type_entity2=doc_real["entities"][relation["tail"]]["type"]
            relations_real[str(relation_pos)] = (relation_words, relation["type"].lower(), type_entity1.lower(),type_entity2.lower())

        for relation in data_preds[doc_id]["relations"]:
            relation_pos= (data_preds[doc_id]["entities"][relation["head"]]["start"], data_preds[doc_id]["entities"][relation["head"]]["end"]
                           ,data_preds[doc_id]["entities"][relation["tail"]]["start"],data_preds[doc_id]["entities"][relation["tail"]]["end"])
            relation_words = (' '.join(doc_real["tokens"][data_preds[doc_id]["entities"][relation["head"]]["start"]:data_preds[doc_id]["entities"][relation["head"]]["end"]]),
                   ' '.join(doc_real["tokens"][data_preds[doc_id]["entities"][relation["tail"]]["start"]:data_preds[doc_id]["entities"][relation["tail"]]["end"]]))
            relation_probs = relation["probs"].split("[")[1].split("]")[0]
            probs =np.fromstring(relation_probs, dtype=float, sep=' ')
            prob =probs[relations_types.index(relation['type'])]
            
            entity1 = data_preds[doc_id]["entities"][relation["head"]]
            entity2 = data_preds[doc_id]["entities"][relation["tail"]]
            entity_probs = entity1["probs"].split("[")[1].split("]")[0]
            probs =np.fromstring(entity_probs, dtype=float, sep=' ')
            probe1 =probs[entities_types.index(entity1['type'])]
            
            entity_probs = entity2["probs"].split("[")[1].split("]")[0]
            probs =np.fromstring(entity_probs, dtype=float, sep=' ')
            probe2 =probs[entities_types.index(entity2['type'])]
            prob = prob*probe1*probe2
            
            type_entity1= entity1["type"]
            type_entity2=entity2["type"]
            relations_pred[str(relation_pos)] = (prob, relation_words, relation["type"].lower(), type_entity1.lower(),type_entity2.lower())

        for idx in relations_real:
            if idx not in relations_pred:
                relations_pred[idx]=(0,"0***")
        for idx in relations_pred:
            if idx not in relations_real:
                relations_real[idx]=(0,"0***")
        display_real=[]
        display_pred =[]
        positions=[]
        for idx in relations_real :
            display_real.append(relations_real[idx])
            display_pred.append(relations_pred[idx])
            positions.append(idx)
            longnr+=1
            if relations_real[idx]!=relations_pred[idx][1:] and relations_real[idx][1]!="0***"  and relations_pred[idx][1]!="0***" :
                longnr3+=1
            if relations_real[idx]!=relations_pred[idx][1:] and  relations_pred[idx][1]=="0***" :
                longnr2+=1  # il n'identifie pas l'entité
            if relations_real[idx]!=relations_pred[idx][1:] and  relations_real[idx][1]=="0***" :
                longnr1+=1  # il identifie l'entité 
                
            
            
            
            

            """if relations_pred[idx] !=  relations_real[idx] and  relations_real[idx]=="0***"  :
                display_real.append(relations_real[idx])
                display_pred.append(relations_pred[idx])
                positions.append(idx)"""



        df ={"position":positions , "real":display_real , "predictions":display_pred}
        df =pd.DataFrame(data=df)
        print("doc n°:",doc_id)
        print(df)
        print("(", longn3,"**",longn2,"**",longn1,"**",longn,"**) (",longr3,"**",longr2,"**",longr1,"**",longr,"**) (",longnr3,"**",longnr2,"**",longnr1,"**",longnr,")" "****************************************************separation***********************")
        
study(data_reals,data_preds)

"""
print("**************************************Separtation")
thresholds=[0.4]
#print("logs\spert_treshold_"+str(threshold)+"/*.json")
for threshold in thresholds:
    files =glob.glob("CONLL04/logs/spert_treshold_0.4/neurASP-SUP_*.json")
    for file_ in files :
        print(file_)    
        with open(file_) as file :
            data_neur  = json.load(file)
        study(data_reals,data_preds)
        print (file_)
       """

<class 'module'>
doc n°: 0
   position                     real                            predictions
0    (5, 7)  (Hakawati Theatre, org)  (0.96285242, (Hakawati Theatre, org))
1    (8, 9)            (Arab, other)                              (0, 0***)
2  (10, 11)         (Jerusalem, loc)         (0.99789667, (Jerusalem, loc))
3  (17, 18)    (Palestinians, other)      (0.713116, (Palestinians, other))
doc n°: 0
         position                                          real predictions
0  (5, 7, 10, 11)  ((Hakawati Theatre, Jerusalem), orgbased_in)   (0, 0***)
doc n°: 0
         position                                               real predictions
0  (5, 7, 10, 11)  ((Hakawati Theatre, Jerusalem), orgbased_in, o...   (0, 0***)
( 0 ** 1 ** 0 ** 4 **) ( 0 ** 1 ** 0 ** 1 **) ( 0 ** 1 ** 0 ** 1 )****************************************************separation***********************
doc n°: 1
  position            real                   predictions
0   (0, 1)  (PERUGIA, loc)  (0.99778938,

1  (16, 17, 4, 5)       ((Khrushchev, Soviet), live_in, peop, loc)  (0.7592301251993104, (Khrushchev, Soviet), liv...
( 0 ** 9 ** 6 ** 40 **) ( 0 ** 4 ** 0 ** 11 **) ( 0 ** 4 ** 0 ** 11 )****************************************************separation***********************
doc n°: 8
   position                real                       predictions
0    (0, 1)     (Mikoyan, peop)     (0.99776447, (Mikoyan, peop))
1    (4, 5)  (Khrushchev, peop)  (0.98293316, (Khrushchev, peop))
2  (13, 14)       (Soviet, loc)        (0.9817878, (Soviet, loc))
doc n°: 8
         position                             real predictions
0  (0, 1, 13, 14)     ((Mikoyan, Soviet), live_in)   (0, 0***)
1  (4, 5, 13, 14)  ((Khrushchev, Soviet), live_in)   (0, 0***)
doc n°: 8
         position                                        real predictions
0  (0, 1, 13, 14)     ((Mikoyan, Soviet), live_in, peop, loc)   (0, 0***)
1  (4, 5, 13, 14)  ((Khrushchev, Soviet), live_in, peop, loc)   (0, 0***)
( 0 ** 9 ** 6 ** 43 **

( 0 ** 13 ** 9 ** 91 **) ( 0 ** 10 ** 6 ** 43 **) ( 0 ** 10 ** 6 ** 43 )****************************************************separation***********************
doc n°: 22
   position                                  real                                        predictions
0  (18, 20)                 (Donald Miller, peop)                (0.99497366, (Donald Miller, peop))
1  (23, 27)  (United Federation of Teachers, org)  (0.98198342, (United Federation of Teachers, o...
2  (28, 30)                   (Bert Shanas, peop)                  (0.99341148, (Bert Shanas, peop))
3  (28, 31)                             (0, 0***)                (0.98969424, (Bert Shanas ., peop))
doc n°: 22
           position  ...                                        predictions
0  (28, 30, 23, 27)  ...  (0.99988031, (Bert Shanas, United Federation o...
1  (18, 20, 23, 27)  ...  (0.49798551, (Donald Miller, United Federation...
2  (28, 31, 23, 27)  ...  (0.99987221, (Bert Shanas ., United Federation...

[3 rows x 

1    (4, 5, 18, 19)                (0, 0***)  (0.95611161, (Nixon, U.S.), live_in)
doc n°: 34
           position                                real                                        predictions
0  (23, 24, 18, 19)  ((Bush, U.S.), live_in, peop, loc)  (0.6174335732157177, (Bush, U.S.), live_in, pe...
1    (4, 5, 18, 19)                           (0, 0***)  (0.9457906377393602, (Nixon, U.S.), live_in, p...
( 1 ** 17 ** 15 ** 146 **) ( 0 ** 13 ** 17 ** 75 **) ( 1 ** 13 ** 17 ** 75 )****************************************************separation***********************
doc n°: 35
   position                                      real                                        predictions
0  (12, 13)                            (Reagan, peop)                       (0.98456776, (Reagan, peop))
1  (14, 17)                   (Ralph K. Winter, peop)              (0.98788983, (Ralph K. Winter, peop))
2  (19, 25)  (2nd U.S. Circuit Court of Appeals, org)  (0.97146553, (2nd U.S. Circuit Court of Ap

1  (10, 11, 26, 27)  ((Navy, U.S.), orgbased_in, org, loc)  (0.5275239097063529, (Navy, U.S.), orgbased_in...
( 2 ** 18 ** 16 ** 174 **) ( 1 ** 13 ** 19 ** 88 **) ( 2 ** 13 ** 19 ** 88 )****************************************************separation***********************
doc n°: 42
   position                  real                           predictions
0    (3, 4)           (U.S., loc)             (0.99445081, (U.S., loc))
1  (31, 33)    (Arab League, org)      (0.99249458, (Arab League, org))
2  (34, 36)  (Chedli Klibi, peop)    (0.99220771, (Chedli Klibi, peop))
3  (34, 37)             (0, 0***)  (0.98656589, (Chedli Klibi ., peop))
doc n°: 42
           position                                     real                                        predictions
0  (34, 36, 31, 33)  ((Chedli Klibi, Arab League), work_for)  (0.99988103, (Chedli Klibi, Arab League), work...
1  (34, 37, 31, 33)                                (0, 0***)  (0.99986267, (Chedli Klibi ., Arab League), wo...
doc n°: 42

...

( 11 ** 95 ** 119 ** 1074 **) ( 3 ** 135 ** 114 ** 501 **) ( 4 ** 135 ** 114 ** 501 )****************************************************separation***********************
doc n°: 254
   position                     real                            predictions
0    (0, 1)             (Dole, peop)              (0.9148074, (Dole, peop))
1    (4, 6)    (North Carolina, loc)    (0.99505782, (North Carolina, loc))
2  (24, 26)   (Elizabeth Dole, peop)   (0.99581808, (Elizabeth Dole, peop))
3  (31, 34)  (Salisbury , N.C., loc)  (0.97955132, (Salisbury , N.C., loc))
4  (40, 41)             (Dole, peop)             (0.99557257, (Dole, peop))
5  (59, 60)             (Bush, peop)             (0.99746466, (Bush, peop))
6  (11, 12)                (0, 0***)         (0.7680284, (Southern, other))
doc n°: 254
           position                                           real                                        predictions
0  (24, 26, 31, 34)  ((Elizabeth Dole, Salisbury , N.C.), live_in)  (0.99920744

2  (10, 13)  (Whitingham , Vt, loc)  (0.9461872, (Whitingham , Vt, loc))
doc n°: 260
         position                                         real                                        predictions
0  (5, 7, 10, 13)  ((Brigham Young, Whitingham , Vt), live_in)  (0.99881184, (Brigham Young, Whitingham , Vt),...
doc n°: 260
         position                                               real                                        predictions
0  (5, 7, 10, 13)  ((Brigham Young, Whitingham , Vt), live_in, pe...  (0.9367444204746294, (Brigham Young, Whitingha...
( 11 ** 99 ** 123 ** 1109 **) ( 3 ** 135 ** 118 ** 512 **) ( 4 ** 135 ** 118 ** 512 )****************************************************separation***********************
doc n°: 261
   position                 real                        predictions
0    (0, 1)       (Martin, peop)       (0.99496788, (Martin, peop))
1    (2, 3)        (Jimbob, loc)       (0.5209335, (Jimbob, other))
2  (19, 20)       (Rogers, peop)       (0.995903

3  (25, 26)                (Lubbock, loc)                (0.99943095, (Lubbock, loc))
doc n°: 267
           position                         real                                        predictions
0  (21, 22, 25, 26)  ((Holly, Lubbock), live_in)            (0.99003124, (Holly, Lubbock), live_in)
1  (17, 20, 21, 22)                    (0, 0***)  (0.50144452, (Texas Tech University, Holly), o...
doc n°: 267
           position                                    real                                        predictions
0  (21, 22, 25, 26)  ((Holly, Lubbock), live_in, peop, loc)  (0.9858874930509726, (Holly, Lubbock), live_in...
1  (17, 20, 21, 22)                               (0, 0***)  (0.49480141835613456, (Texas Tech University, ...
( 12 ** 102 ** 125 ** 1140 **) ( 3 ** 136 ** 123 ** 524 **) ( 4 ** 136 ** 123 ** 524 )****************************************************separation***********************
doc n°: 268
   position                  real                        predictions
0   

4  (37, 40)          (Denison , Tex, loc)         (0.98348039, (Denison , Tex, loc))
doc n°: 283
           position  ...                                        predictions
0  (19, 22, 37, 40)  ...  (0.982556224, (Dwight D. Eisenhower, Denison ,...

[1 rows x 3 columns]
doc n°: 283
           position  ...                                        predictions
0  (19, 22, 37, 40)  ...  (0.9591339691706368, (Dwight D. Eisenhower, De...

[1 rows x 3 columns]
( 13 ** 103 ** 127 ** 1193 **) ( 3 ** 137 ** 126 ** 543 **) ( 4 ** 137 ** 126 ** 543 )****************************************************separation***********************
doc n°: 284
   position                          real                                 predictions
0    (0, 2)           (Bette Davis, peop)           (0.99519712, (Bette Davis, peop))
1    (4, 7)  (Ruth Elizabeth Davis, peop)  (0.98840034, (Ruth Elizabeth Davis, peop))
2    (8, 9)                (April, other)                (0.98072964, (April, other))
3  (14, 17)    

'\nprint("**************************************Separtation")\nthresholds=[0.4]\n#print("logs\\spert_treshold_"+str(threshold)+"/*.json")\nfor threshold in thresholds:\n    files =glob.glob("CONLL04/logs/spert_treshold_0.4/neurASP-SUP_*.json")\n    for file_ in files :\n        print(file_)    \n        with open(file_) as file :\n            data_neur  = json.load(file)\n        study(data_reals,data_preds)\n        print (file_)\n       '

In [2]:
study(data_preds,data_neur)


doc n°: 0
   position                     real                            predictions
0    (5, 7)  (Hakawati Theatre, org)  (0.96285242, (Hakawati Theatre, org))
1  (10, 11)         (Jerusalem, loc)         (0.99789667, (Jerusalem, loc))
2  (17, 18)    (Palestinians, other)      (0.713116, (Palestinians, other))
doc n°: 0
Empty DataFrame
Columns: [position, real, predictions]
Index: []
doc n°: 0
Empty DataFrame
Columns: [position, real, predictions]
Index: []
( 0 ** 0 ** 0 ** 3 **) ( 0 ** 0 ** 0 ** 0 **) ( 0 ** 0 ** 0 ** 0 )****************************************************separation***********************
doc n°: 1
  position            real                   predictions
0   (0, 1)  (PERUGIA, loc)  (0.99778938, (PERUGIA, loc))
1   (2, 3)    (Italy, loc)    (0.99636394, (Italy, loc))
2   (4, 5)       (AP, org)       (0.99744898, (AP, org))
doc n°: 1
       position                            real                                 predictions
0  (0, 1, 2, 3)  ((PERUGIA, Italy), located_

3  (24, 25)                    (Earth, loc)                    (0.95072877, (Earth, loc))
doc n°: 17
       position                                               real                                        predictions
0  (3, 6, 0, 2)  ((Michael D. Papagiannis, Boston University), ...  (0.99861705, (Michael D. Papagiannis, Boston U...
doc n°: 17
       position                                               real                                        predictions
0  (3, 6, 0, 2)  ((Michael D. Papagiannis, Boston University), ...  (0.99861705, (Michael D. Papagiannis, Boston U...
( 1 ** 2 ** 0 ** 64 **) ( 0 ** 0 ** 0 ** 23 **) ( 0 ** 0 ** 0 ** 23 )****************************************************separation***********************
doc n°: 18
   position                   real                          predictions
0    (5, 6)         (Ostined, loc)         (0.98952019, (Ostined, loc))
1    (7, 8)           (Lesny, loc)           (0.99264538, (Lesny, loc))
2   (9, 10)         (Liberec, loc)

4  (20, 22)     (El Toro, loc)     (0.99490756, (El Toro, loc))
doc n°: 24
         position                                 real                                   predictions
0  (20, 22, 5, 6)  ((El Toro, California), located_in)  (0.42535, (El Toro, California), located_in)
doc n°: 24
         position                                 real                                   predictions
0  (20, 22, 5, 6)  ((El Toro, California), located_in)  (0.42535, (El Toro, California), located_in)
( 1 ** 4 ** 0 ** 89 **) ( 0 ** 4 ** 2 ** 40 **) ( 0 ** 4 ** 2 ** 40 )****************************************************separation***********************
doc n°: 25
   position                real                      predictions
0    (5, 7)     (2 a.m., other)     (0.8091954, (2 a.m., other))
1  (15, 16)  (Yellowstone, loc)  (0.9962464, (Yellowstone, loc))
2  (17, 18)         (Wyo., loc)        (0.99821448, (Wyo., loc))
3  (23, 24)      (McAllen, loc)     (0.99551386, (McAllen, loc))
4  (25, 26)        

0  (9, 11, 12, 14)  ((Darryl Breniser, Blue Ball), work_for)  (0.94165295, (Darryl Breniser, Blue Ball), wor...
doc n°: 37
          position                                      real                                        predictions
0  (9, 11, 12, 14)  ((Darryl Breniser, Blue Ball), work_for)  (0.94165295, (Darryl Breniser, Blue Ball), wor...
( 1 ** 5 ** 0 ** 143 **) ( 0 ** 9 ** 4 ** 72 **) ( 0 ** 9 ** 4 ** 72 )****************************************************separation***********************
doc n°: 38
   position                        real                               predictions
0    (0, 2)         (Terry Hands, peop)         (0.99532735, (Terry Hands, peop))
1  (47, 48)  (Stratford-upon-Avon, loc)  (0.99430841, (Stratford-upon-Avon, loc))
2  (54, 55)               (London, loc)               (0.99514848, (London, loc))
3  (58, 59)             (Barbican, loc)             (0.99686807, (Barbican, loc))
doc n°: 38
           position                              real            

1  (12, 15, 17, 18)    ((Rodolfo Neri Vela, Mexican), live_in)  (0.47527146, (Rodolfo Neri Vela, Mexican), liv...
doc n°: 51
           position                                       real                                        predictions
0     (6, 8, 9, 11)  ((Sultan Al-Saud, Saudi Arabia), live_in)  (0.99117494, (Sultan Al-Saud, Saudi Arabia), l...
1  (12, 15, 17, 18)    ((Rodolfo Neri Vela, Mexican), live_in)  (0.47527146, (Rodolfo Neri Vela, Mexican), liv...
( 1 ** 7 ** 0 ** 201 **) ( 1 ** 19 ** 9 ** 105 **) ( 1 ** 19 ** 9 ** 105 )****************************************************separation***********************
doc n°: 52
   position                    real                           predictions
0   (9, 11)  (Rocky Mountains, loc)  (0.99007142, (Rocky Mountains, loc))
1  (12, 13)          (Montana, loc)           (0.9971891, (Montana, loc))
2  (19, 21)         (50 mph, other)         (0.99017745, (50 mph, other))
3  (22, 23)       (Livingston, loc)       (0.99862599, (Livingston

...

0  (7, 10, 0, 4)  ((Sirhan B. Sirhan, Sen. Robert F. Kennedy), k...  (0.9949097, (Sirhan B. Sirhan, Sen. Robert F. ...
doc n°: 223
        position                                               real                                        predictions
0  (7, 10, 0, 4)  ((Sirhan B. Sirhan, Sen. Robert F. Kennedy), k...  (0.9949097, (Sirhan B. Sirhan, Sen. Robert F. ...
( 8 ** 30 ** 0 ** 875 **) ( 2 ** 70 ** 33 ** 367 **) ( 2 ** 70 ** 33 ** 367 )****************************************************separation***********************
doc n°: 224
  position                           real                                  predictions
0   (0, 1)                (Kennedy, peop)                (0.99604452, (Kennedy, peop))
1   (4, 7)  (Sirhan Bishara Sirhan, peop)  (0.98091298, (Sirhan Bishara Sirhan, peop))
doc n°: 224
       position                                      real                                        predictions
0  (4, 7, 0, 1)  ((Sirhan Bishara Sirhan, Kennedy), kill)  (0.99943084, (S

1  (14, 18)  (Martin Luther King Jr., peop)  (0.98345745, (Martin Luther King Jr., peop))
doc n°: 231
         position                                              real                                        predictions
0  (0, 3, 14, 18)  ((James Earl Ray, Martin Luther King Jr.), kill)  (0.99936634, (James Earl Ray, Martin Luther Ki...
doc n°: 231
         position                                              real                                        predictions
0  (0, 3, 14, 18)  ((James Earl Ray, Martin Luther King Jr.), kill)  (0.99936634, (James Earl Ray, Martin Luther Ki...
( 8 ** 30 ** 0 ** 904 **) ( 2 ** 70 ** 33 ** 377 **) ( 2 ** 70 ** 33 ** 377 )****************************************************separation***********************
doc n°: 232
   position                    real                           predictions
0    (3, 6)  (James Earl Ray, peop)  (0.98895139, (James Earl Ray, peop))
1   (9, 10)          (Memphis, loc)          (0.78174603, (Memphis, loc))
2  (11, 12)  

( 13 ** 31 ** 0 ** 1034 **) ( 2 ** 73 ** 35 ** 421 **) ( 2 ** 73 ** 35 ** 421 )****************************************************separation***********************
doc n°: 267
   position                          real                                 predictions
0  (10, 12)           (Buddy Holly, peop)          (0.18136114, (Buddy Holly, other))
1  (17, 20)  (Texas Tech University, org)  (0.99033558, (Texas Tech University, org))
2  (21, 22)                 (Holly, peop)                 (0.99638152, (Holly, peop))
3  (25, 26)                (Lubbock, loc)                (0.99943095, (Lubbock, loc))
doc n°: 267
           position                                           real                              predictions
0  (17, 20, 21, 22)  ((Texas Tech University, Holly), orgbased_in)                                (0, 0***)
1  (21, 22, 25, 26)                    ((Holly, Lubbock), live_in)  (0.99003124, (Holly, Lubbock), live_in)
doc n°: 267
           position                          

4  (35, 38)          (Vienna , Austria, loc)          (0.97838086, (Vienna , Austria, loc))
doc n°: 275
           position  ...                                        predictions
0  (16, 19, 23, 26)  ...  (0.99739069, (Martin Van Buren, Kinderhook , N...

[1 rows x 3 columns]
doc n°: 275
           position  ...                                        predictions
0  (16, 19, 23, 26)  ...  (0.99739069, (Martin Van Buren, Kinderhook , N...

[1 rows x 3 columns]
( 14 ** 31 ** 0 ** 1062 **) ( 2 ** 74 ** 35 ** 431 **) ( 2 ** 74 ** 35 ** 431 )****************************************************separation***********************
doc n°: 276
   position                   real                          predictions
0    (7, 8)          (Jan., other)          (0.99406034, (Jan., other))
1  (17, 19)  (Elvis Presley, peop)  (0.99611086, (Elvis Presley, peop))
2  (22, 25)   (Tupelo , Miss, loc)   (0.97435635, (Tupelo , Miss, loc))
doc n°: 276
           position                                       r

4  (37, 40)          (Denison , Tex, loc)         (0.98348039, (Denison , Tex, loc))
doc n°: 283
           position  ...                                        predictions
0  (19, 22, 37, 40)  ...  (0.982556224, (Dwight D. Eisenhower, Denison ,...

[1 rows x 3 columns]
doc n°: 283
           position  ...                                        predictions
0  (19, 22, 37, 40)  ...  (0.982556224, (Dwight D. Eisenhower, Denison ,...

[1 rows x 3 columns]
( 15 ** 32 ** 0 ** 1090 **) ( 2 ** 76 ** 36 ** 442 **) ( 2 ** 76 ** 36 ** 442 )****************************************************separation***********************
doc n°: 284
   position                          real                                 predictions
0    (0, 2)           (Bette Davis, peop)           (0.99519712, (Bette Davis, peop))
1    (4, 7)  (Ruth Elizabeth Davis, peop)  (0.98840034, (Ruth Elizabeth Davis, peop))
2    (8, 9)                (April, other)                (0.98072964, (April, other))
3  (14, 17)          (

In [ ]:
with open("CONLL04\conll04_test.json") as file :
    data_reals = json.load(file)
files =glob.glob("CONLL04/logs/spert_treshold_0.4/neurASP-SUP_predictions*.json")
for file_ in files :
    print(file_)
    with open(file_) as file :
        data_preds = json.load(file)
    print(file_)
    study(data_reals,data_preds)
"""

print(type(json))
with open("CONLL04/logs/spert_treshold_0.4/neurASP-SUP_predictions_valid_epoch_20.json") as file :
    data_reals = json.load(file)
with open("CONLL04/logs/spert_treshold_0.4/predictions_valid_epoch_20.json") as file :
    #Scierc/logs/scierc_treshold_0.1/predictions_valid_epoch_1.json
    data_preds = json.load(file)
    
study(data_reals,data_preds)"""

In [ ]:
### enlever la classe no aussi sur les predictions 

In [46]:

import numpy as np   
import cv2
import matplotlib.pyplot as plt
import os 
from matplotlib import *
import torchvision
import sys
sys.path.append('../../')
import time
import torch
from network import Net
from neurasp import NeurASP
import pandas as pd
from torchvision import datasets, transforms
import helper
import cv2
import glob
import json
import unicodedata

#### method 3: we don't fix anything but we can have entity moving and we add a confidence to entities since it has a higher confidence in prediction

print("***********************************************METHOD3")

m = Net()    
nnMapping = {'nerjoinre': m}  

def NeurSUP (data_JSON,file_result,sup=False, sup_Entity=False):

    if sup_Entity :
        FEdprogram = r'''
        nn(nerjoinre(1,R,IdR), [ work_For, kill, orgBased_In, live_In, located_In,no]) :- relation(R,IdR,IdE1,IdE2).
        nn(nerjoinre(1,E,IdE), [no, loc, org, peop, other]) :- entity(E,IdE,Value,Start,End).

        '''
        FEaspProgram = r'''
        entity(E,IdE,Value) :- entity(E,IdE,Value,Start,End).
        
        % we remove near span duplication 
        1{nerjoinre(0,E,IdE1,no);nerjoinre(0,E,IdE2,no) }1  :-  entity(E,IdE1,_,Start1,End1),entity(E,IdE2,_,Start2,End2), (|Start1 - Start2|+|End1-End2|)=1.
        nested(IdE1, IdE2)  :- nerjoinre(0,E,IdE1,Name), Name!=no, entity(E,IdE1,_,Start1,End1),entity(E,IdE2,_,Start2,End2), (|Start1 - Start2|+|End1-End2|)=1.
        
        % knowledge transfer 
        newrelation(IdR,IdE3,IdE1,Relation) ,nerjoinre(0,R,IdR,no) :- nested(IdE1, IdE2), entity(E,IdE1,_), entity(E,IdE2,_), relation(R,IdR,IdE3,IdE1),nerjoinre(0,R,IdR,Relation), Relation!=no.
        newrelation(IdR,IdE1,IdE3,Relation) ,nerjoinre(0,R,IdR,no) :- nested(IdE1, IdE2), entity(E,IdE1,_), entity(E,IdE2,_), relation(R,IdR,IdE1,IdE3),nerjoinre(0,R,IdR,Relation), Relation!=no.
        
        % identical entities should have the same relation if none is null of curse ***** à tester sans : 69,89 sans et 70,17 avec
        %newrelation(IdR,IdE2,IdE3,Relation) :- relation(R,IdR,IdE1,IdE3),nerjoinre(0,R,IdR,Relation), nerjoinre(0,E,IdE1,Named),nerjoinre(0,E,IdE2,Named), Named!=no, Relation!=no, entity(E,IdE1,Value), entity(E,IdE2,Value).
        %newrelation(IdR,IdE3,IdE2,Relation) :- relation(R,IdR,IdE3,IdE1),nerjoinre(0,R,IdR,Relation), nerjoinre(0,E,IdE1,Named),nerjoinre(0,E,IdE2,Named), Named!=no,Relation!=no, entity(E,IdE1,Value), entity(E,IdE2,Value).

        % dependencies
            % dependencies in location
        newrelation(IdE1,IdE1,IdE3,located_In) :- relation(R,IdR1,IdE1,IdE2),relation(R,IdR2,IdE2,IdE3),not relation(R,_,IdE1,IdE3), nerjoinre(0,E,IdE3,loc), nerjoinre(0,E,IdE2,loc), nerjoinre(0,E,IdE1,loc),nerjoinre(0,R,IdR1,located_In),nerjoinre(0,R,IdR2,located_In).
   
        % an entity can't have two types  
         
         test(Value, Named) :- nerjoinre(0,E,IdE,Named) , entity(E,IdE,Value), not nerjoinre(0,E,IdE,no).
         test_1(Value) :- entity(E,IdE,Value).
         count_value(Value, S) :- S=#count{E : test(Value, E)} ,test_1(Value).
         :- count_value(Value, S), S>1 .       
         
         
        % if a relation has an entity no, it should be assign no directly. 
         nerjoinre(0,R,IdR,no):-relation(R,IdR,IdE1,IdE2), entity(E,IdE1,_), nerjoinre(0,E,IdE1,no).
         nerjoinre(0,R,IdR,no):-relation(R,IdR,IdE1,IdE2), entity(E,IdE2,_), nerjoinre(0,E,IdE2,no).

        % the type assignment to each entity should be consistent with it's neighbour relation

         group(IdR,R1,E1,E2):- relation(R,IdR,IdE1,IdE2),nerjoinre(0,E,IdE1,E1), nerjoinre(0,E,IdE2,E2),nerjoinre(0,R,IdR,R1).
         :- relation(R,IdR,IdE1,IdE2),nerjoinre(0,E,IdE2,E2),nerjoinre(0,E,IdE1,E1), entity(e,IdE1,_), entity(e,IdE2,_), not group(IdR,no,E1,E2), 
         not group(IdR,work_For,peop,org), not group(IdR,kill,peop,peop), not group(IdR,orgBased_In,org,loc),
        not group(IdR,live_In,peop,loc), not group(IdR,located_In,loc,loc) .
        
        % type assignement for new relations
        groupNew(IdR,R1,E1,E2):- newrelation(IdR,IdE1,IdE2,R1),nerjoinre(0,E,IdE1,E1), nerjoinre(0,E,IdE2,E2).

         :- newrelation(IdR,IdE1,IdE2,R1),not groupNew(IdR,work_For,peop,org), not groupNew(IdR,kill,peop,peop),
         not groupNew(IdR,orgBased_In,org,loc),
        not groupNew(IdR,live_In,peop,loc), not groupNew(IdR,located_In,loc,loc) .
        
        '''

    #i add relation(R,IdR,IdE1,IdE2), entity(e,IdE1,_,E1), entity(e,IdE2,_,E2), not group(IdR,no,E1,E2) from the previous test
 
    
    for doc_id , document in enumerate(data_JSON):
        #document =data_JSON[1]
        
        Factlist=""
        DataList={}
        entities=[]
        entities_with_all=document["entities"]
        for idx, entity_brut in enumerate(entities_with_all):
            entity_words = (''.join(document["tokens"][entity_brut["start"]:entity_brut["end"]])).replace("ó","o")
            text = unicodedata.normalize('NFD', entity_words)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
            entity_words = str(text)
            entity_probs = entity_brut["probs"].split("[")[1].split("]")[0]
            entity_type = str(entity_brut["type"])
            Factlist=Factlist+"entity(e,e"+str(idx)+",'b_"+(''.join(e for e in entity_words if e.isalnum())).lower()+"',"+str(entity_brut["start"])+","+str(entity_brut["end"])+").\n "
            probs =1*np.fromstring(entity_probs, dtype=float, sep=' ')
            probs[0]=0.1
            #print("********",probs)
            DataList['e,e'+str(idx)]=torch.tensor([probs], dtype=torch.float64)

            
            #**********to be back *********DataList['e,e'+str(idx)]=torch.tensor([probs], dtype=torch.float64)

        # relation(r,r1,e1,e2)ZZ
        relations_with_all=document["relations"]
        for idx ,relation_brut in enumerate(relations_with_all):
            #print("**rel**",relation_brut)
            relation_probs = relation_brut["probs"].split("[")[1].split("]")[0]
            probs= np.fromstring(relation_probs, dtype=float, sep=' ')
            probs =np.append(probs, [0.3])
            DataList['r,r'+str(idx)] = torch.tensor([probs], dtype=torch.float64)
            Factlist=Factlist+"relation(r,r"+str(idx)+",e"+str(relation_brut["head"])+",e"+str(relation_brut["tail"])+").\n "
        Factlist=[Factlist]
        DataList=[DataList]
        #print("datalist***",DataList)
        #print("factlist***",Factlist)

        for idx, facts in enumerate(Factlist):
            # Initialize NeurASP object
            
            NeurASPobj = NeurASP(FEdprogram + facts, nnMapping, optimizers=None)
            # Find the most probable stable model
            models = NeurASPobj.infer(dataDic=DataList[idx], obs='', mvpp=FEaspProgram + facts)
            print("**",models)
        bool=True
        if bool :
            for model in models[0]:
                for atom in model:
                    if 'nerjoinre(0,r,r' in atom:
                        relation_id= atom.split("nerjoinre(0,r,r")[1].split(",")[0]
                        data_JSON[doc_id]["relations"][int(relation_id)]["type"]= atom.split(",")[3].split(")")[0].upper()[:1] + atom.split(",")[3].split(")")[0][1:]

                    if 'newrelation(' in atom:
                        relation={}
                        relation["head"]=int(atom.split(",")[1][1])
                        relation["tail"]=int(atom.split(",")[2][1])
                        relation["type"]=atom.split(",")[3].split(")")[0][:1].upper() + atom.split(",")[3].split(")")[0][1:]
                        print (relation)
                        data_JSON[doc_id]["relations"].append(relation)
                        
                    if 'nerjoinre(0,e,e' in atom :
                        entity_id= atom.split("nerjoinre(0,e,e")[1].split(",")[0]
                        data_JSON[doc_id]["entities"][int(entity_id)]["type"]= atom.split(",")[3].split(")")[0][:1].upper() + atom.split(",")[3].split(")")[0][1:]

            if sup_Entity:
                entity_with_all =[]
                removed=0
                for idx,entity in enumerate(data_JSON[doc_id]["entities"]):
                    removed+=1
                    if entity["type"]!="No":
                        entity_with_all.append(entity)
                    else :
                        
                        for relation in data_JSON[doc_id]["relations"] :
                            if relation["head"]>= removed:
                                relation["head"]-=1
                            if relation["tail"]>= removed:
                                relation["tail"]-=1
                        removed-=1
                data_JSON[doc_id]["entities"]=entity_with_all
                
            if sup :
                relation_with_all =[]
                for relation in data_JSON[doc_id]["relations"]:
                    if relation["type"]!="No":
                        relation_with_all.append(relation)

                data_JSON[doc_id]["relations"]=relation_with_all

    with open(file_result, 'w') as f:
        json.dump(data_JSON, f)
        print("*********** You can actually test your model with evaluation functions")

print("ok")
thresholds=[0.4]
#print("logs\spert_treshold_"+str(threshold)+"/*.json")
for threshold in thresholds:
    files =glob.glob("CONLL04\logs\spert_treshold_"+str(threshold)+"\predictions*.json")
    for file_ in files :
        file = "CONLL04\logs\spert_treshold_0.4\prepredictions_valid_epoch_16.json"
        with open(file_) as file :
            data_JSON = json.load(file)
        file_result=file_.split("\\")[0] +"\\"+file_.split("\\")[1] +"\\"+file_.split("\\")[2]
        file_name=file_.split('\\')[-1]
        file_result3=file_result+"\\neurASP-SUP_"+file_name
        NeurSUP(data_JSON,file_result3,sup=True,sup_Entity=True)
        break 


***********************************************METHOD3
ok
pi_prime 
nn(nerjoinre(1,R,IdR), ( work_For, kill, orgBased_In, live_In, located_In,no)) :- relation(R,IdR,IdE1,IdE2).
nn(nerjoinre(1,E,IdE), (no, loc, org, peop, other)) :- entity(E,IdE,Value,Start,End).

entity(e,e0,'b_hakawatitheatre',5,7).
entity(e,e1,'b_jerusalem',10,11).
entity(e,e2,'b_palestinians',17,18).


end of the pi_prime
*************** 1{nerjoinre(0,e,e0,no); nerjoinre(0,e,e0,loc); nerjoinre(0,e,e0,org); nerjoinre(0,e,e0,peop); nerjoinre(0,e,e0,other)}1.
1{nerjoinre(0,e,e1,no); nerjoinre(0,e,e1,loc); nerjoinre(0,e,e1,org); nerjoinre(0,e,e1,peop); nerjoinre(0,e,e1,other)}1.
1{nerjoinre(0,e,e2,no); nerjoinre(0,e,e2,loc); nerjoinre(0,e,e2,org); nerjoinre(0,e,e2,peop); nerjoinre(0,e,e2,other)}1.

entity(E,IdE,Value) :- entity(E,IdE,Value,Start,End).

% we remove near span duplication
1{nerjoinre(0,E,IdE1,no);nerjoinre(0,E,IdE2,no) }1  :-  entity(E,IdE1,_,Start1,End1),entity(E,IdE2,_,Start2,End2), (|Start1 - Start2|+|E


** ([['nerjoinre(0,r,r2,orgBased_In)', 'nerjoinre(0,e,e0,loc)', 'nerjoinre(0,e,e1,loc)', 'nerjoinre(0,e,e2,org)', 'nerjoinre(0,r,r0,located_In)', 'nerjoinre(0,r,r1,orgBased_In)', "entity(e,e0,'b_perugia',0,1)", "entity(e,e1,'b_italy',2,3)", "entity(e,e2,'b_ap',4,5)", "entity(e,e0,'b_perugia')", "entity(e,e1,'b_italy')", "entity(e,e2,'b_ap')", 'relation(r,r0,e0,e1)', 'relation(r,r1,e2,e0)', 'relation(r,r2,e2,e1)', "test('b_perugia',loc)", "test('b_italy',loc)", "test('b_ap',org)", "test_1('b_perugia')", "test_1('b_italy')", "test_1('b_ap')", "count_value('b_perugia',1)", "count_value('b_italy',1)", "count_value('b_ap',1)", 'group(r0,located_In,loc,loc)', 'group(r1,orgBased_In,org,loc)', 'group(r2,orgBased_In,org,loc)']], '')
pi_prime 
nn(nerjoinre(1,R,IdR), ( work_For, kill, orgBased_In, live_In, located_In,no)) :- relation(R,IdR,IdE1,IdE2).
nn(nerjoinre(1,E,IdE), (no, loc, org, peop, other)) :- entity(E,IdE,Value,Start,End).

entity(e,e0,'b_reagan',0,1).
entity(e,e1,'b_america',32,33)


** ([['nerjoinre(0,e,e5,peop)', 'nerjoinre(0,e,e3,no)', 'nerjoinre(0,e,e0,org)', 'nerjoinre(0,e,e1,loc)', 'nerjoinre(0,e,e2,other)', 'nerjoinre(0,e,e4,other)', "entity(e,e0,'b_congress',15,16)", "entity(e,e1,'b_washington',33,34)", "entity(e,e2,'b_dec13',35,37)", "entity(e,e3,'b_dec13',35,38)", "entity(e,e4,'b_dec13reagan',35,39)", "entity(e,e5,'b_reagan',38,39)", "entity(e,e0,'b_congress')", "entity(e,e1,'b_washington')", "entity(e,e2,'b_dec13')", "entity(e,e3,'b_dec13')", "entity(e,e4,'b_dec13reagan')", "entity(e,e5,'b_reagan')", 'nested(e4,e3)', 'nested(e2,e3)', "test('b_congress',org)", "test('b_washington',loc)", "test('b_dec13',other)", "test('b_dec13reagan',other)", "test('b_reagan',peop)", "test_1('b_congress')", "test_1('b_washington')", "test_1('b_dec13')", "test_1('b_dec13reagan')", "test_1('b_reagan')", "count_value('b_congress',1)", "count_value('b_washington',1)", "count_value('b_dec13',1)", "count_value('b_dec13reagan',1)", "count_value('b_reagan',1)"]], '')
pi_prime 
n


** ([['nerjoinre(0,r,r0,work_For)', 'nerjoinre(0,e,e0,peop)', 'nerjoinre(0,e,e1,org)', 'nerjoinre(0,e,e2,peop)', 'nerjoinre(0,e,e3,other)', 'nerjoinre(0,e,e4,loc)', "entity(e,e0,'b_wright',0,1)", "entity(e,e1,'b_universityoftexas',3,6)", "entity(e,e2,'b_nixon',13,14)", "entity(e,e3,'b_watergate',22,23)", "entity(e,e4,'b_whitehouse',30,32)", "entity(e,e0,'b_wright')", "entity(e,e1,'b_universityoftexas')", "entity(e,e2,'b_nixon')", "entity(e,e3,'b_watergate')", "entity(e,e4,'b_whitehouse')", 'relation(r,r0,e0,e1)', "test('b_wright',peop)", "test('b_universityoftexas',org)", "test('b_nixon',peop)", "test('b_watergate',other)", "test('b_whitehouse',loc)", "test_1('b_wright')", "test_1('b_universityoftexas')", "test_1('b_nixon')", "test_1('b_watergate')", "test_1('b_whitehouse')", "count_value('b_wright',1)", "count_value('b_universityoftexas',1)", "count_value('b_nixon',1)", "count_value('b_watergate',1)", "count_value('b_whitehouse',1)", 'group(r0,work_For,peop,org)']], '')
pi_prime 
nn(


** ([['nerjoinre(0,r,r1,live_In)', 'nerjoinre(0,e,e0,peop)', 'nerjoinre(0,e,e1,loc)', 'nerjoinre(0,e,e2,peop)', 'nerjoinre(0,r,r0,live_In)', "entity(e,e0,'b_anastasmikoyan',0,2)", "entity(e,e1,'b_soviet',4,5)", "entity(e,e2,'b_khrushchev',16,17)", "entity(e,e0,'b_anastasmikoyan')", "entity(e,e1,'b_soviet')", "entity(e,e2,'b_khrushchev')", 'relation(r,r0,e0,e1)', 'relation(r,r1,e2,e1)', "test('b_anastasmikoyan',peop)", "test('b_soviet',loc)", "test('b_khrushchev',peop)", "test_1('b_anastasmikoyan')", "test_1('b_soviet')", "test_1('b_khrushchev')", "count_value('b_anastasmikoyan',1)", "count_value('b_soviet',1)", "count_value('b_khrushchev',1)", 'group(r0,live_In,peop,loc)', 'group(r1,live_In,peop,loc)']], '')
pi_prime 
nn(nerjoinre(1,R,IdR), ( work_For, kill, orgBased_In, live_In, located_In,no)) :- relation(R,IdR,IdE1,IdE2).
nn(nerjoinre(1,E,IdE), (no, loc, org, peop, other)) :- entity(E,IdE,Value,Start,End).

entity(e,e0,'b_mikoyan',0,1).
entity(e,e1,'b_khrushchev',4,5).
entity(e,e2,

...


** ([['nerjoinre(0,r,r0,live_In)', 'nerjoinre(0,e,e0,peop)', 'nerjoinre(0,e,e1,loc)', 'nerjoinre(0,e,e2,loc)', "entity(e,e0,'b_benjaminharrison',7,9)", "entity(e,e1,'b_unitedstates',15,17)", "entity(e,e2,'b_northbendohio',21,25)", "entity(e,e0,'b_benjaminharrison')", "entity(e,e1,'b_unitedstates')", "entity(e,e2,'b_northbendohio')", 'relation(r,r0,e0,e2)', "test('b_benjaminharrison',peop)", "test('b_unitedstates',loc)", "test('b_northbendohio',loc)", "test_1('b_benjaminharrison')", "test_1('b_unitedstates')", "test_1('b_northbendohio')", "count_value('b_benjaminharrison',1)", "count_value('b_unitedstates',1)", "count_value('b_northbendohio',1)", 'group(r0,live_In,peop,loc)']], '')
pi_prime 
nn(nerjoinre(1,R,IdR), ( work_For, kill, orgBased_In, live_In, located_In,no)) :- relation(R,IdR,IdE1,IdE2).
nn(nerjoinre(1,E,IdE), (no, loc, org, peop, other)) :- entity(E,IdE,Value,Start,End).

entity(e,e0,'b_us',3,4).
entity(e,e1,'b_oliverhazardperry',6,9).
entity(e,e2,'b_southkingstownri',12,16


** ([['nerjoinre(0,r,r0,live_In)', 'nerjoinre(0,e,e0,loc)', 'nerjoinre(0,e,e1,peop)', 'nerjoinre(0,e,e2,loc)', "entity(e,e0,'b_unitedstates',8,10)", "entity(e,e1,'b_rutherfordbhayes',11,14)", "entity(e,e2,'b_delawareohio',18,21)", "entity(e,e0,'b_unitedstates')", "entity(e,e1,'b_rutherfordbhayes')", "entity(e,e2,'b_delawareohio')", 'relation(r,r0,e1,e2)', "test('b_unitedstates',loc)", "test('b_rutherfordbhayes',peop)", "test('b_delawareohio',loc)", "test_1('b_unitedstates')", "test_1('b_rutherfordbhayes')", "test_1('b_delawareohio')", "count_value('b_unitedstates',1)", "count_value('b_rutherfordbhayes',1)", "count_value('b_delawareohio',1)", 'group(r0,live_In,peop,loc)']], '')
pi_prime 
nn(nerjoinre(1,R,IdR), ( work_For, kill, orgBased_In, live_In, located_In,no)) :- relation(R,IdR,IdE1,IdE2).
nn(nerjoinre(1,E,IdE), (no, loc, org, peop, other)) :- entity(E,IdE,Value,Start,End).

entity(e,e0,'b_unitedstates',8,10).
entity(e,e1,'b_jamesgarfield',11,13).
entity(e,e2,'b_orangeohio',17,20)


** ([['nerjoinre(0,r,r0,live_In)', 'nerjoinre(0,e,e0,other)', 'nerjoinre(0,e,e1,loc)', 'nerjoinre(0,e,e2,peop)', 'nerjoinre(0,e,e3,other)', 'nerjoinre(0,e,e4,loc)', "entity(e,e0,'b_photo',0,1)", "entity(e,e1,'b_color',2,3)", "entity(e,e2,'b_dwightdeisenhower',19,22)", "entity(e,e3,'b_oct',25,26)", "entity(e,e4,'b_denisontex',37,40)", "entity(e,e0,'b_photo')", "entity(e,e1,'b_color')", "entity(e,e2,'b_dwightdeisenhower')", "entity(e,e3,'b_oct')", "entity(e,e4,'b_denisontex')", 'relation(r,r0,e2,e4)', "test('b_photo',other)", "test('b_color',loc)", "test('b_dwightdeisenhower',peop)", "test('b_oct',other)", "test('b_denisontex',loc)", "test_1('b_photo')", "test_1('b_color')", "test_1('b_dwightdeisenhower')", "test_1('b_oct')", "test_1('b_denisontex')", "count_value('b_photo',1)", "count_value('b_color',1)", "count_value('b_dwightdeisenhower',1)", "count_value('b_oct',1)", "count_value('b_denisontex',1)", 'group(r0,live_In,peop,loc)']], '')
pi_prime 
nn(nerjoinre(1,R,IdR), ( work_For, kil


** ([['nerjoinre(0,r,r0,live_In)', 'nerjoinre(0,e,e0,other)', 'nerjoinre(0,e,e1,peop)', 'nerjoinre(0,e,e2,loc)', "entity(e,e0,'b_oct',1,2)", "entity(e,e1,'b_jessealexanderhelmsjr',6,10)", "entity(e,e2,'b_monroe',19,20)", "entity(e,e0,'b_oct')", "entity(e,e1,'b_jessealexanderhelmsjr')", "entity(e,e2,'b_monroe')", 'relation(r,r0,e1,e2)', "test('b_oct',other)", "test('b_jessealexanderhelmsjr',peop)", "test('b_monroe',loc)", "test_1('b_oct')", "test_1('b_jessealexanderhelmsjr')", "test_1('b_monroe')", "count_value('b_oct',1)", "count_value('b_jessealexanderhelmsjr',1)", "count_value('b_monroe',1)", 'group(r0,live_In,peop,loc)']], '')
pi_prime 
nn(nerjoinre(1,R,IdR), ( work_For, kill, orgBased_In, live_In, located_In,no)) :- relation(R,IdR,IdE1,IdE2).
nn(nerjoinre(1,E,IdE), (no, loc, org, peop, other)) :- entity(E,IdE,Value,Start,End).

entity(e,e0,'b_house',1,2).
entity(e,e1,'b_lawrencewelk',11,13).
entity(e,e2,'b_northdakota',14,16).
relation(r,r0,e1,e2).


end of the pi_prime
**********


** ([['nerjoinre(0,r,r0,located_In)', 'nerjoinre(0,e,e0,loc)', 'nerjoinre(0,e,e1,loc)', "entity(e,e0,'b_indiana',1,2)", "entity(e,e1,'b_indianapolis',11,12)", "entity(e,e0,'b_indiana')", "entity(e,e1,'b_indianapolis')", 'relation(r,r0,e1,e0)', "test('b_indiana',loc)", "test('b_indianapolis',loc)", "test_1('b_indiana')", "test_1('b_indianapolis')", "count_value('b_indiana',1)", "count_value('b_indianapolis',1)", 'group(r0,located_In,loc,loc)']], '')
*********** You can actually test your model with evaluation functions


In [9]:
 """

with open("Scierc/scierc_train_dev.json") as file :
    data_JSON1 = json.load(file)
    
with open("Scierc/scierc_test.json") as file :
    data_JSON2 = json.load(file)
    
    
def relations (data_JSON):
    relation_type=[]
    for document in data_JSON :
        entities_with_all=document["entities"]
        # relation(r,r1,e1,e2)
        relations_with_all=document["relations"]
        for idx ,relation_brut in enumerate(relations_with_all):
            temp= decapitalize(relation_brut["type"]).replace('-','')+","+decapitalize(entities_with_all[relation_brut["head"]]["type"])+","+decapitalize(entities_with_all[relation_brut["tail"]]["type"])
            if temp not in relation_type:
                relation_type.append(",not group(IdR,"+temp+")")
                #print("temp",temp)
    return relation_type
        
    
    
rel1 =relations (data_JSON1)
rel2 = relations (data_JSON2)
relation =list(set(rel1) | set(rel2))  
relation= ''.join(relation)
#print(relation)

def retreive_relation(relation) :
    relations= [ "Used-for", "Feature-of", "Hyponym-of", "Evaluate-for", "Part-of" , "Compare", "Conjunction","no"]
    for real in relations:
        if relation.lower()==real.replace('-','').lower():
            return real
"""

#i add relation(R,IdR,IdE1,IdE2), entity(e,IdE1,_,E1), entity(e,IdE2,_,E2), not group(IdR,no,E1,E2) from the previous test










## cleaning  ******************** 
#****************************************************************************************************************************
# les tests se font maintenant de façon automatique avec automated_evaluation.py

# we clean a files to have great NER 
import numpy as np
import glob
import json
from ast import literal_eval
def decapitalize(str):
    return str[:1].lower() + str[1:]
  

thresholds=[0.4]
with open("conll04\conll04_test.json") as file :
    data_real = json.load(file)
types =["loc","org", "peop","other"]
#[ "task", "method", "material", "otherScientificTerm","metric","generic"]         


#print("logs\spert_treshold_"+str(threshold)+"/*.json") 
for threshold in thresholds:
    files =glob.glob("conll04\logs\spert_treshold_"+str(threshold)+"\predictions*.json")
    for file_ in files :
        #file_ ="Scierc/logs/scierc_treshold_0.1/predictions_valid_epoch_1.json"
        with open(file_) as file :
            data_JSON = json.load(file)   
            print("ok")
        for doc_id, document in enumerate(data_JSON) :
            entities=document["entities"]
            for idx, entity in enumerate(entities):
                flag =True 
                for real_id, real_entity in enumerate(data_real[doc_id]["entities"])  :
                    if (entity["start"]== real_entity["start"]) and (entity["end"]== real_entity["end"]):
                        entity_probs = entity["probs"].split("[")[1].split("]")[0]
                        probs =np.fromstring(entity_probs, dtype=float, sep=' ')
                        t=len(probs)
                        #print(probs)
                        data_JSON[doc_id]["entities"][idx]["type"]=real_entity["type"]
                        probs =np.random.rand(t) * 0.1 
                        probs[1+ types.index(decapitalize(real_entity["type"]))] = 0.9
                        #print( types.index(decapitalize(real_entity["type"])))
                        data_JSON[doc_id]["entities"][idx]["probs"]=str(probs.tolist()).replace(',','')
                        flag =False 
                if flag:
                    data_JSON[doc_id]["entities"][idx]["type"]="no"
                    entity_probs = entity["probs"].split("[")[1].split("]")[0]
                    probs =np.fromstring(entity_probs, dtype=float, sep=' ')
                    probs =np.random.rand(len(probs)) * 0.01 
                    probs[0] = 0.9
                    data_JSON[doc_id]["entities"][idx]["probs"]=str(probs.tolist()).replace(',','')
        #print(data_JSON)

        with open(file_, 'w') as f:
            json.dump(data_JSON, f)
                #print("*********** You can actually test your model with evaluation functions")




                

ok
ok
ok
